# Research 02 (Unified): BTC TMO + IBIT AMMA vs Combo + Portfolio

This notebook **combines** the previous Research-02 and IBIT-AMMA notebooks into one workflow.

## What it does
1. Rebuilds the Research-02 Trend+Mining+OU (TMO) selection logic on BTC.
2. Applies that selected TMO model to IBIT and compares vs AMMA and Buy&Hold.
3. Builds a no-leverage 5-model IBIT combo (`AMMA`, `Trend`, `Mining`, `OU`, `ZScore`) and optimizes Sharpe.
4. Runs a portfolio sleeve analysis using your provided universe config (with graceful fallback to locally available data).


In [ ]:
# 1) experiment config (as provided)
import datetime

universe = [
  "SPY-US", "SLV-US", "GLD-US", "TLT-US", "USO-US", "UNG-US", "IXJ-US",
  "KXI-US", "JXI-US", "IXG-US", "IXN-US", "RXI-US", "MXI-US", "EXI-US",
  "IXC-US", "IEI-US", "SHY-US", "BIL-US", "JPXN-US", "INDA-US", "MCHI-US",
  "EZU-US", "IBIT-US", "ETHA-US", "VIXY-US"
]
features = ["close_momentum_10"]
models   = ["RXI_TLT_pml_10", "GLD_USO_nml_10"]
aggregators = ["model_mvo"]
optimizers   = ["mean_variance_constrained"]
initial_value = 1_000_000
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2025, 1, 1)

print('Configured universe size:', len(universe))
print('Date range:', start_date, '->', end_date)


In [ ]:
# Imports + path discovery
from pathlib import Path
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

cwd = Path.cwd().resolve()
crypto_root = None
for c in [cwd, cwd / 'Crypto', cwd.parent, cwd.parent / 'Crypto', cwd.parent.parent / 'Crypto']:
    if (c / 'Data' / 'IBIT_Data.csv').exists():
        crypto_root = c
        break
if crypto_root is None:
    raise FileNotFoundError('Could not locate Crypto root with Data/IBIT_Data.csv')

quant_root = crypto_root.parent
if str(crypto_root) not in sys.path:
    sys.path.insert(0, str(crypto_root))

import config as cfg
from Backtest.amma import amma_from_ibit_csv
from Data.raw_data_loader import load_raw_crypto_csv, load_ibit_with_mining_cost
from Models.trend import trend_signal
from Models.mining import mining_signal
from Models.ou import ou_signal
from Models.zscore import zscore_signal


In [ ]:
# Core utilities

def run_backtest_from_position(price: pd.Series, pos: pd.Series, fee_bps: float, slippage_bps: float, long_only: bool = True) -> dict:
    price = price.astype(float)
    ret = price.pct_change().fillna(0.0)
    pos = pos.reindex(price.index).fillna(0.0)
    if long_only:
        pos = pos.clip(0.0, 1.0)
    turnover = pos.diff().abs().fillna(pos.abs())
    costs = turnover * ((fee_bps + slippage_bps) / 1e4)
    net_ret = (pos * ret) - costs
    net_equity = (1.0 + net_ret).cumprod()
    return {'net_ret': net_ret, 'net_equity': net_equity, 'turnover': turnover, 'pos': pos}


def sr(x: pd.Series, periods: int = 252) -> float:
    x = x.dropna()
    if len(x) < 10:
        return np.nan
    s = x.std(ddof=1)
    if s <= 0 or not np.isfinite(s):
        return np.nan
    return float(np.sqrt(periods) * x.mean() / s)


def sortino(x: pd.Series, periods: int = 252) -> float:
    x = x.dropna()
    dn = x[x < 0]
    if len(dn) < 2:
        return np.nan
    s = dn.std(ddof=1)
    if s <= 0 or not np.isfinite(s):
        return np.nan
    return float((x.mean() * periods) / (s * np.sqrt(periods)))


def cagr(x: pd.Series, periods: int = 252) -> float:
    eq = (1.0 + x.fillna(0.0)).cumprod()
    if len(eq) < 2:
        return np.nan
    years = len(eq) / periods
    return float(eq.iloc[-1] ** (1.0 / years) - 1.0) if years > 0 else np.nan


def maxdd(x: pd.Series) -> float:
    eq = (1.0 + x.fillna(0.0)).cumprod()
    dd = eq / eq.cummax() - 1.0
    return float(dd.min())


def ann_vol(x: pd.Series, periods: int = 252) -> float:
    x = x.dropna()
    return float(np.sqrt(periods) * x.std(ddof=1)) if len(x) > 1 else np.nan


def win_rate(x: pd.Series) -> float:
    x = x.dropna()
    return float((x > 0).mean()) if len(x) else np.nan


def metric_row(name: str, r: pd.Series) -> dict:
    return {
        'Strategy': name,
        'CAGR': cagr(r),
        'Sharpe': sr(r),
        'Sortino': sortino(r),
        'MaxDD': maxdd(r),
        'Volatility': ann_vol(r),
        'WinRate': win_rate(r),
    }


## A) Systematic Research-02 fix: rebuild exact BTC TMO model selection

In [ ]:
# BTC data for TMO model selection (Trend + Mining + OU)

df_btc = load_raw_crypto_csv(cfg.DATA_PATH, start_date=cfg.DATA_START_DATE)
price_btc = df_btc[cfg.PRICE_COLUMN_BTC].astype(float)

trend_pos_btc = trend_signal(
    df_btc,
    price_column=cfg.PRICE_COLUMN_BTC,
    fast_window=cfg.TREND_FAST_WINDOW,
    slow_window=cfg.TREND_SLOW_WINDOW,
    long_only=True,
    leverage_aggressive=1.0,
    leverage_neutral=0.5,
    leverage_defensive=0.0,
).reindex(price_btc.index).fillna(0.0).clip(0.0, 1.0)

mining_pos_btc = mining_signal(
    df_btc,
    z_window=cfg.MINING_Z_WINDOW,
    entry_z=cfg.MINING_ENTRY_Z,
    exit_z=cfg.MINING_EXIT_Z,
    use_log_edge=cfg.MINING_USE_LOG_EDGE,
).reindex(price_btc.index).fillna(0.0).clip(0.0, 1.0)

ou_pos_btc = ou_signal(
    price_btc,
    window=cfg.OU_WINDOW,
    entry_z=cfg.OU_ENTRY_Z,
    exit_z=max(cfg.OU_EXIT_Z, 0.3),
    long_short=False,
).reindex(price_btc.index).fillna(0.0).clip(0.0, 1.0)

# Exact Research-02 FINAL-cell style: 0.5/0.5 Trend+Mining then OU blend sweep
trend_mining_pos = (0.5 * trend_pos_btc + 0.5 * mining_pos_btc).clip(0.0, 1.0)
trend_mining_results = run_backtest_from_position(price_btc, trend_mining_pos, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)

alphas = np.linspace(0.0, 1.0, 41)
best_tmo = {
    'alpha_ou': 0.0,
    'sr': sr(trend_mining_results['net_ret']),
    'pos': trend_mining_pos,
    'results': trend_mining_results,
    'label': 'Trend+Mining (0.5/0.5)'
}
for a in alphas:
    p = ((1 - a) * trend_mining_pos + a * ou_pos_btc).clip(0.0, 1.0)
    r = run_backtest_from_position(price_btc, p, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)
    s = sr(r['net_ret'])
    if np.isfinite(s) and s > best_tmo['sr'] + 1e-6:
        best_tmo = {'alpha_ou': float(a), 'sr': float(s), 'pos': p, 'results': r, 'label': f'Trend+Mining+OU (a={a:.2f})'}

print('Selected BTC TMO model:', best_tmo['label'])
print('Selected alpha_ou:', best_tmo['alpha_ou'])
print('Selected Sharpe:', best_tmo['sr'])


## B) IBIT AMMA vs TMO (from Research-02) + Buy&Hold

In [ ]:
# Load IBIT panel with mining cost

panel_ibit = load_ibit_with_mining_cost(
    str(crypto_root / 'Data' / 'IBIT_Data.csv'),
    str(crypto_root / 'Data' / 'cleaned_crypto_data.csv'),
    forward_fill_mining_cost=True,
).sort_index().copy()
panel_ibit['close'] = panel_ibit['close'].astype(float)

model_ibit = pd.DataFrame(index=panel_ibit.index)
model_ibit['BTC-USD_close'] = panel_ibit['close']
model_ibit['COST_TO_MINE'] = panel_ibit['mining_cost']

# AMMA
amma_df = amma_from_ibit_csv(
    str(crypto_root / 'Data' / 'IBIT_Data.csv'),
    momentum_weights={5:0.25, 10:0.25, 20:0.25, 60:0.25},
    threshold=0.0,
    long_enabled=True,
    short_enabled=False,
).rename(columns={'Date':'date'}).set_index('date')
amma_pos = amma_df.reindex(panel_ibit.index)['amma_position'].fillna(0.0).clip(0.0, 1.0)

# TMO components directly on IBIT panel
trend_pos_ibit = trend_signal(
    model_ibit,
    price_column='BTC-USD_close',
    fast_window=cfg.TREND_FAST_WINDOW,
    slow_window=cfg.TREND_SLOW_WINDOW,
    long_only=True,
    leverage_aggressive=1.0,
    leverage_neutral=0.5,
    leverage_defensive=0.0,
).reindex(panel_ibit.index).fillna(0.0).clip(0.0, 1.0)

mining_pos_ibit = mining_signal(
    model_ibit,
    z_window=cfg.MINING_Z_WINDOW,
    entry_z=cfg.MINING_ENTRY_Z,
    exit_z=cfg.MINING_EXIT_Z,
    use_log_edge=cfg.MINING_USE_LOG_EDGE,
).reindex(panel_ibit.index).fillna(0.0).clip(0.0, 1.0)

ou_pos_ibit = ou_signal(
    model_ibit['BTC-USD_close'],
    window=cfg.OU_WINDOW,
    entry_z=cfg.OU_ENTRY_Z,
    exit_z=max(cfg.OU_EXIT_Z, 0.3),
    long_short=False,
).reindex(panel_ibit.index).fillna(0.0).clip(0.0, 1.0)

# Apply BTC-selected alpha to IBIT TMO
trend_mining_ibit = (0.5 * trend_pos_ibit + 0.5 * mining_pos_ibit).clip(0.0, 1.0)
tmo_pos_ibit = ((1 - best_tmo['alpha_ou']) * trend_mining_ibit + best_tmo['alpha_ou'] * ou_pos_ibit).clip(0.0, 1.0)

# Backtests
bh_bt = run_backtest_from_position(panel_ibit['close'], pd.Series(1.0, index=panel_ibit.index), 0.0, 0.0)
amma_bt = run_backtest_from_position(panel_ibit['close'], amma_pos, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)
tmo_bt = run_backtest_from_position(panel_ibit['close'], tmo_pos_ibit, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)

ret_ibit = pd.DataFrame({
    'BuyHold_IBIT': bh_bt['net_ret'],
    'AMMA_IBIT': amma_bt['net_ret'],
    'TMO_IBIT': tmo_bt['net_ret'],
}, index=panel_ibit.index)

table_ibit = pd.DataFrame([
    metric_row('BuyHold_IBIT', ret_ibit['BuyHold_IBIT']),
    metric_row('AMMA_IBIT', ret_ibit['AMMA_IBIT']),
    metric_row('TMO_IBIT', ret_ibit['TMO_IBIT']),
]).set_index('Strategy').sort_values('Sharpe', ascending=False)
table_ibit


In [ ]:
# IBIT equity + drawdown plot (AMMA orange)

eq_ibit = (1.0 + ret_ibit).cumprod()
dd_ibit = eq_ibit / eq_ibit.cummax() - 1.0

plt.figure(figsize=(12,5))
plt.plot(eq_ibit.index, eq_ibit['TMO_IBIT'], label='TMO (Research-02)', linewidth=2.4, color='tab:blue')
plt.plot(eq_ibit.index, eq_ibit['AMMA_IBIT'], label='AMMA (orange line)', linewidth=2.2, color='tab:orange')
plt.plot(eq_ibit.index, eq_ibit['BuyHold_IBIT'], label='Buy & Hold', linewidth=1.8, linestyle='--', color='tab:gray')
plt.title('IBIT Equity: TMO vs AMMA vs Buy&Hold')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

plt.figure(figsize=(12,4))
plt.plot(dd_ibit.index, dd_ibit['TMO_IBIT'], label='TMO (Research-02)', linewidth=2.0, color='tab:blue')
plt.plot(dd_ibit.index, dd_ibit['AMMA_IBIT'], label='AMMA (orange line)', linewidth=2.0, color='tab:orange')
plt.plot(dd_ibit.index, dd_ibit['BuyHold_IBIT'], label='Buy & Hold', linewidth=1.5, linestyle='--', color='tab:gray')
plt.title('IBIT Drawdown: TMO vs AMMA')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


## C) No-leverage 5-model combo on IBIT (must target AMMA outperformance)

In [ ]:
# Build 5-model positions on IBIT and optimize no-leverage weights

zscore_pos_ibit = zscore_signal(
    model_ibit['BTC-USD_close'],
    resid_window=120,
    entry_z=1.25,
    exit_z=0.4,
    long_short=False,
    use_vol_target=False,
    max_leverage=1.0,
).reindex(panel_ibit.index).fillna(0.0).clip(0.0, 1.0)

pos5 = pd.DataFrame({
    'AMMA': amma_pos,
    'Trend': trend_pos_ibit,
    'Mining': mining_pos_ibit,
    'OU': ou_pos_ibit,
    'ZScore': zscore_pos_ibit,
}, index=panel_ibit.index)

models5 = ['AMMA','Trend','Mining','OU','ZScore']
ret5 = {}
for m in models5:
    ret5[m] = run_backtest_from_position(panel_ibit['close'], pos5[m], cfg.FEE_BPS, cfg.SLIPPAGE_BPS)['net_ret']
ret5 = pd.DataFrame(ret5, index=panel_ibit.index).dropna()

amma_sharpe = sr(ret5['AMMA'])

rng = np.random.default_rng(123)
candidates = [
    np.array([1,0,0,0,0], float),
    np.array([0,1,0,0,0], float),
    np.array([0,0,1,0,0], float),
    np.array([0,0,0,1,0], float),
    np.array([0,0,0,0,1], float),
    np.ones(5)/5,
]
for _ in range(180000):
    candidates.append(rng.dirichlet(np.ones(5)))

best5 = {'w': np.array([1,0,0,0,0], float), 'sharpe': -np.inf, 'ret': ret5['AMMA']}
for w in candidates:
    r = (ret5[models5] * w).sum(axis=1)
    s = sr(r)
    if np.isfinite(s) and s > best5['sharpe']:
        best5 = {'w': w, 'sharpe': float(s), 'ret': r}

w5 = pd.Series(best5['w'], index=models5)
combo5_pos = (pos5[models5] * w5.values).sum(axis=1).clip(0.0, 1.0)
combo5_bt = run_backtest_from_position(panel_ibit['close'], combo5_pos, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)
combo5_sharpe = sr(combo5_bt['net_ret'])

print('Sharpe(AMMA):', amma_sharpe)
print('Sharpe(5-model combo):', combo5_sharpe)
print('weights:')
print(w5)

if combo5_sharpe > amma_sharpe:
    print('SUCCESS: 5-model combo Sharpe beats AMMA in this in-sample run.')
else:
    print('WARNING: 5-model combo did not beat AMMA Sharpe in this run; retune parameters/search depth.')


In [ ]:
# Compare AMMA vs 5-model combo vs buyhold (focus: orange AMMA line)

ret_cmp = pd.DataFrame({
    'BuyHold_IBIT': bh_bt['net_ret'],
    'AMMA_IBIT': amma_bt['net_ret'],
    'Combo5_IBIT': combo5_bt['net_ret'],
}, index=panel_ibit.index)

eq_cmp = (1.0 + ret_cmp).cumprod()

plt.figure(figsize=(12,5))
plt.plot(eq_cmp.index, eq_cmp['Combo5_IBIT'], label='Combo5 (optimized no-leverage)', linewidth=2.5, color='tab:blue')
plt.plot(eq_cmp.index, eq_cmp['AMMA_IBIT'], label='AMMA (orange line)', linewidth=2.2, color='tab:orange')
plt.plot(eq_cmp.index, eq_cmp['BuyHold_IBIT'], label='Buy & Hold', linewidth=1.8, linestyle='--', color='tab:gray')
plt.title('IBIT Equity: Optimized 5-model combo vs AMMA')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

table_cmp = pd.DataFrame([
    metric_row('BuyHold_IBIT', ret_cmp['BuyHold_IBIT']),
    metric_row('AMMA_IBIT', ret_cmp['AMMA_IBIT']),
    metric_row('Combo5_IBIT', ret_cmp['Combo5_IBIT']),
]).set_index('Strategy').sort_values('Sharpe', ascending=False)

table_cmp


## D) Portfolio sleeve check with your universe config

In [ ]:
# Use locally available data from your universe config for a practical sleeve test

def parse_price_csv(path: Path) -> pd.Series:
    x = pd.read_csv(path, encoding='utf-8-sig')
    x['Date'] = pd.to_datetime(x['Date'], format='%m/%d/%y', errors='coerce')
    px = pd.to_numeric(
        x['Price'].astype(str).str.replace(',', '', regex=False).str.replace('$', '', regex=False),
        errors='coerce'
    )
    return pd.Series(px.values, index=x['Date']).dropna().sort_index()

local_file_map = {
    'SPY-US': quant_root / 'Market Data' / 'Equity ETF' / 'SPY.csv',
    'IBIT-US': quant_root / 'Market Data' / 'Crypto Data' / 'IBIT.csv',
    'ETHA-US': quant_root / 'Market Data' / 'Crypto Data' / 'ETHA.csv',
}

available = {k:v for k,v in local_file_map.items() if v.exists()}
missing_from_universe = [u for u in universe if u not in available]
print('Available from configured universe in local data:', sorted(available.keys()))
print('Missing in local dataset count:', len(missing_from_universe))

asset_ret = {}
for t,p in available.items():
    s = parse_price_csv(p)
    asset_ret[t] = s.pct_change()
asset_ret = pd.DataFrame(asset_ret).dropna(how='all')

# Inject IBIT sleeves
port_df = asset_ret.join(pd.DataFrame({
    'IBIT_AMMA': amma_bt['net_ret'],
    'IBIT_Combo5': combo5_bt['net_ret'],
}, index=panel_ibit.index), how='inner').dropna()

# Baseline: SPY + IBIT_AMMA; New: SPY + IBIT_Combo5; Combined: SPY + both

if 'SPY-US' in port_df.columns:
    p_base = 0.7 * port_df['SPY-US'] + 0.3 * port_df['IBIT_AMMA']
    p_new  = 0.7 * port_df['SPY-US'] + 0.3 * port_df['IBIT_Combo5']
    p_both = 0.7 * port_df['SPY-US'] + 0.15 * port_df['IBIT_AMMA'] + 0.15 * port_df['IBIT_Combo5']
else:
    # fallback if SPY unavailable
    p_base = port_df['IBIT_AMMA']
    p_new  = port_df['IBIT_Combo5']
    p_both = 0.5 * port_df['IBIT_AMMA'] + 0.5 * port_df['IBIT_Combo5']

port_table = pd.DataFrame([
    metric_row('Portfolio_Baseline_AMMA', p_base),
    metric_row('Portfolio_Replace_With_Combo5', p_new),
    metric_row('Portfolio_Keep_Both', p_both),
]).set_index('Strategy').sort_values('Sharpe', ascending=False)

corr_amma_combo = float(port_df['IBIT_AMMA'].corr(port_df['IBIT_Combo5']))
print('Correlation(IBIT_AMMA, IBIT_Combo5)=', corr_amma_combo)
port_table


In [ ]:
# Final direct answer with hard metrics

print('=== IBIT strategy table (AMMA vs TMO vs Combo5) ===')
print(pd.concat([table_ibit, table_cmp]).groupby(level=0).first().sort_values('Sharpe', ascending=False))

print('\n=== Portfolio sleeve table ===')
print(port_table)

sA = table_cmp.loc['AMMA_IBIT', 'Sharpe']
sC = table_cmp.loc['Combo5_IBIT', 'Sharpe']
dA = table_cmp.loc['AMMA_IBIT', 'MaxDD']
dC = table_cmp.loc['Combo5_IBIT', 'MaxDD']

print('\n=== Decision ===')
if sC > sA:
    print('Combo5 improves risk-adjusted return vs AMMA (higher Sharpe).')
else:
    print('Combo5 does NOT improve Sharpe vs AMMA in this run.')

if dC >= dA:
    print('Combo5 drawdown is similar or better than AMMA.')
else:
    print('Combo5 drawdown is worse than AMMA.')

print('Use these printed tables for replace/combine decision in portfolio allocation.')
